In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
import os
from PIL import Image
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Load data from image files


In [18]:
def image_to_data(image_folder='/content/drive/MyDrive/dcgan/Landscape'):
    target_size = 512
    data = []
    count = 0
    for filename in os.listdir(image_folder):
        if count >= 500:
            break
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path)
        width, height = image.size
        max_left = width - target_size
        max_top = height - target_size
        if max_left < 0 or max_top < 0:
            continue
        left = random.randint(0, max_left)
        top = random.randint(0, max_top)
        right = left + target_size
        bottom = top + target_size
        cropped_image = image.crop((left, top, right, bottom))
        resized_image = cropped_image.resize((target_size, target_size))
        image_array = np.array(resized_image)
        if len(image_array.shape) < 3 or image_array.shape[-1] != 3:
            continue
        data.append(image_array[np.newaxis, :, :, :])

        count += 1
    data = np.concatenate(data, axis=0)
    return data

X_train = image_to_data()
X_train = (X_train.astype('float32')) / 255

# Build a DCGAN


In [19]:
def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Conv2D(8, kernel_size=4, strides=2, padding="same", input_shape=(512, 512, 3)))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(16, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(32, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(8 * 8 * 512, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 512)))
    model.add(layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="sigmoid"))
    return model

discriminator = build_discriminator()
generator = build_generator(5000)

generator_optimizer = keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.5)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.5)

# Define loss function

In [20]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define a function to show and save images generated during training

In [21]:
def show_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, :])
        plt.axis('off')

    plt.savefig(f"training_log_img/epoch_{epoch + 1}.png")
    plt.show()
    plt.clf()

# Training function

In [22]:
@tf.function
def train_step(images):

    noise = tf.random.normal([batch_size, 5000])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Training

In [ ]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(X_train.shape[0]).batch(batch_size)

epochs = 2000
for epoch in range(epochs):
    for image_batch in train_dataset:
        train_step(image_batch)

    if (epoch + 1) % 20 == 0:
        show_and_save_images(generator, epoch, tf.random.normal([16, 5000]))
        print(f"Epoch {epoch + 1}/{epochs}")

/usr/local/lib/python3.10/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
